In [645]:
import pandas as pd 
import tabula 
from datetime import date

In [646]:
today = date.fromisoformat('2020-07-02')

In [647]:
today.day

2

## Read data from file

In [648]:
df_data = pd.read_csv('../Data/data.csv')
df_data

,Data,Ogółem,Ludność,Liczba aktywnych przypadków,Aktywnych na 10tys mieszkańców,Liczba nowych przypadków
0,2020-09-13,1839,779115,527,6.764085,NaN
1,2020-09-03,1552,779115,869,11.153681,-287.0
2,2020-08-31,1480,779115,888,11.397547,-72.0
3,2020-10-01,2365,779115,461,5.916970,885.0


## Get and parse data from WSEE Kraków

### Get data

In [649]:
if(today.month >= 10):
    suffix = ''
    zero = ''
else:
    suffix = '-MPWIS'
    zero = '0'

link_list = [f"https://wsse.krakow.pl/page/wp-content/uploads/{today.year}/{zero}{today.month}/COVID-19-ogniska-stan-na-{today.day}.{zero}{today.month}.{today.year}-MPWIS.pdf",
        f"https://wsse.krakow.pl/page/wp-content/uploads/{today.year}/{zero}{today.month}/COVID-19-ogniska-stan-na-0{today.day}.{zero}{today.month}.{today.year}-MPWIS.pdf",
        f"https://wsse.krakow.pl/page/wp-content/uploads/{today.year}/{zero}{today.month}/COVID-19-ogniska-stan-na-{today.day}.{zero}{today.month}.{today.year}.pdf",
        f"https://wsse.krakow.pl/page/wp-content/uploads/{today.year}/{zero}{today.month}/COVID-19-ogniska-stan-na-0{today.day}.{zero}{today.month}.{today.year}.pdf",
        f"https://wsse.krakow.pl/page/wp-content/uploads/{today.year}/{zero}{today.month}/COVID-19-ogniska-stan-na-0{today.day}.{zero}{today.month}.{today.year}-.pdf"]

for link in link_list:
    try:
        df_tab = tabula.read_pdf(link)
    except:
        continue
    break

try:
    df_tab = pd.DataFrame(df_tab[0])
except KeyError:
    raise NameError(f"Parse failed on {today}")

if today.month < 9:
    df_tab[df_tab['Unnamed: 0'] == 'Ogółem'] = df_tab[df_tab['Unnamed: 0'] == 'Ogółem'].shift(periods = 1, axis = 1)


&#39;pages&#39; argument isn&#39;t specified.Will extract only from page 1 by default.


### Fix improper string parsing

In [650]:
def fix_str(item):
    if isinstance(item, str):
        return item.replace('\r', ' ').replace('1)', '').replace('3)', '')

def fix_series(series):
    return series.apply(fix_str)


df_tab = df_tab.apply(fix_series, axis=0)

### Change labels and trim table

In [651]:
df_tab.iloc[1,0] = df_tab.iloc[0,0]
df_tab.iloc[1,1] = df_tab.iloc[0,1]

df_tab.columns = df_tab.iloc[1]
df_tab.drop(df_tab.index[1], inplace=True)
df_tab.drop(df_tab.index[0], inplace=True)
df_tab[df_tab['Lp.'] == 'Razem Małopolska'] = df_tab[df_tab['Lp.'] == 'Razem Małopolska'].shift(periods = 1, axis = 1)
df_tab

1,Lp.,Powiat,Ogółem,Ludność,Zapadalność / Aktywne przypadki (na 10 000),Liczba zgonów,Liczba wyleczonych,Liczba aktywnych przypadków,Osoby z kwarantanny po powrocie z zagranicy,Osoby z kwarantanny po kontakcie z osobą zakażoną,Zachorowania poprzez kontakt w szpitalu lub przychodni,Zachorowania w DPS,Zachorowania w ZOL,Inne przypadki zachorowań
2,1,bocheński,152,106 858,"14,22",3,139,8,2,65,12,54,0,None
3,2,brzeski,68,93 201,"7,30",2,64,0,2,5,48,0,2,None
4,3,chrzanowski,37,124 536,"2,97",0,23,14,1,18,0,0,0,None
5,4,dąbrowski,15,59 174,"2,53",0,11,4,2,12,1,0,0,None
6,5,gorlicki,10,108 886,"0,92",0,7,3,1,6,1,0,0,None
7,6,krakowski,86,279 239,"3,08",1,63,22,4,38,19,0,0,None
8,7,limanowski,124,131 764,"9,41",3,104,17,0,68,25,0,0,None
9,8,miechowski,12,48 818,"2,46",0,12,0,0,5,0,0,0,None
10,9,myślenicki,159,127 600,"12,46",0,149,10,0,86,18,0,0,None
11,10,nowosądecki,115,216 796,"5,30",0,30,85,7,108,0,0,0,None


In [652]:
df_tab = df_tab[['Powiat', 'Ogółem', 'Ludność', 'Liczba aktywnych przypadków']]

KeyError: &quot;[&#39;Liczba aktywnych przypadków&#39;] not in index&quot;

## Prepare data for Kraków

### Leave only numerical values for Kraków

In [638]:
df_Krk = df_tab[df_tab['Powiat'] == 'm. Kraków']
df_Krk.drop(columns = ['Powiat'], inplace = True)
df_Krk.columns.name = ''

In [639]:
df_Krk['Ogółem'] = [int(str(val).replace(' ','')) for val in df_Krk['Ogółem'].values]
df_Krk['Ludność'] = [int(str(val).replace(' ','')) for val in df_Krk['Ludność'].values]
df_Krk['Liczba aktywnych przypadków'] = [int(str(val).replace(' ','')) for val in df_Krk['Liczba aktywnych przypadków'].values]

,Ogółem,Ludność,Liczba aktywnych przypadków,Data
21,2279,779115,421,2020-10-03


### Perform calculations

In [640]:
df_Krk['Data'] = today
df_Krk['Aktywnych na 10tys mieszkańców'] = (df_Krk['Liczba aktywnych przypadków'] / df_Krk['Ludność']) * 10000
df_Krk

,Ogółem,Ludność,Liczba aktywnych przypadków,Data,Aktywnych na 10tys mieszkańców
21,2365,779115,461,2020-10-01,5.91697


### Append to df_data and save

In [641]:
if str(today) not in df_data['Data'].values:
    df_data = df_data.append(df_Krk)

In [642]:
df_data['Liczba nowych przypadków'] = df_data['Ogółem'].diff().astype(int, errors = 'ignore')

In [643]:
df_data.set_index(df_data['Data'], inplace = True, drop = True)
df_data.drop(columns = ['Data'], inplace = True)
df_data.columns.name = None
df_data

,Ogółem,Ludność,Liczba aktywnych przypadków,Aktywnych na 10tys mieszkańców,Liczba nowych przypadków
Data,,,,,
2020-09-13,1839,779115,527,6.764085,NaN
2020-09-03,1552,779115,869,11.153681,-287.0
2020-08-31,1480,779115,888,11.397547,-72.0
2020-10-01,2365,779115,461,5.916970,885.0


In [644]:
df_data.to_csv('../Data/data.csv')